In [128]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [129]:
!ls ../input

In [130]:
import torch
from torch import nn
from torchvision import datasets, models, transforms


In [131]:
import torch.utils.data as tdata

In [132]:
!ls ../input/seg_train/seg_train/
!ls ../input/seg_test/seg_test/

In [133]:
data_path_format = '../input/seg_{0}/seg_{0}'

In [134]:
np.random.seed(5315)
torch.manual_seed(9784)

In [135]:
# channel_means = (0.4301, 0.4572, 0.4540)
# channel_stds = (0.2698, 0.2680, 0.2983)  # 0.485, 0.456, 0.406
channel_means = (0.485, 0.456, 0.406)
channel_stds = (0.229, 0.224, 0.225)

In [136]:
image_transforms = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize(channel_means, channel_stds)
])
    

In [137]:
image_datasets = dict(zip(('dev', 'test'), [datasets.ImageFolder(data_path_format.format(key),transform=image_transforms) for key in ['train', 'test']]))
print(image_datasets)

In [138]:
devset_indices = np.arange(len(image_datasets['dev']))
devset_labels = image_datasets['dev'].targets

In [139]:
from sklearn import model_selection

In [140]:
train_indices, val_indices, train_labels,  val_labels = model_selection.train_test_split(devset_indices, devset_labels, test_size=0.1, stratify=devset_labels)

In [141]:
print(len(val_indices))

In [142]:
image_datasets['train'] = tdata.Subset(image_datasets['dev'], train_indices)
image_datasets['validation'] = tdata.Subset(image_datasets['dev'], val_indices)

In [143]:
print(image_datasets)

In [144]:
from IPython.display import display

In [145]:
image_dataloaders = {key: tdata.DataLoader(image_datasets[key], batch_size=16,shuffle=True) for key in  ['train', 'validation']}

In [146]:
image_dataloaders['test'] = tdata.DataLoader(image_datasets['test'], batch_size=32)

In [147]:
def imshow(inp, title=None, fig_size=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0)) # C x H x W  # H x W x C
    inp = channel_stds * inp + channel_means
    inp = np.clip(inp, 0, 1)
    fig = plt.figure(figsize=fig_size)
    ax = fig.add_subplot('111')
    ax.imshow(inp)
    if title is not None:
        ax.set_title(title)
    ax.set_aspect('equal')
    plt.pause(0.001)  

In [148]:
imshow(image_datasets['train'][8555][0]) # 5946

In [149]:
cuda_device = torch.device('cuda')
cpu_device = torch.device('cpu')
device = cuda_device

In [150]:
import torch.nn as nn
import torch.nn as F

class MyModel(nn.Module):
    def conv_layer(self, input_, output_, kernel):
            return nn.Sequential(torch.nn.Conv2d(input_, output_, kernel, padding=1), 
                                 nn.Tanh(),
                                 nn.BatchNorm2d(output_),
                                 nn.MaxPool2d(2,2))
    def conv_layer_dropout(self, input_, output_, kernel):
            return nn.Sequential(torch.nn.Conv2d(input_, output_, kernel, padding=1), 
                                 nn.ReLU(),
                                 nn.Dropout(p = 0.15),
                                 nn.BatchNorm2d(output_),
                                 nn.MaxPool2d(2,2))
        
    def linear_layer(self,input_,output_):
            return nn.Sequential(torch.nn.Linear(input_, output_),
                                nn.ReLU())
    
    def __init__(self):
            #150
            super(MyModel, self).__init__()
            self.norm1 = self.conv_layer(3,12,2)
            self.norm2 = self.conv_layer(12,24,2)
            self.norm3 = self.conv_layer_dropout(24,48,2)
            self.norm4 = self.conv_layer_dropout(48,96,2)
            
            self.fc1 = self.linear_layer(96 * 10 * 10,300)
            self.fc2 = self.linear_layer(300, 150)
            self.fc3 = self.linear_layer(150 , 6)

                
    def forward(self, x):
            x = self.norm1(x)
            x = self.norm2(x)
            x = self.norm3(x)
            x = self.norm4(x)
            
            #print(x.size(1) , x.size(2) , x.size(3))
            
            x = x.view(x.size(0),x.size(1) * x.size(2) * x.size(3) )
            x = self.fc1(x)
            x = self.fc2(x)
            x = self.fc3(x)
            return x    
ptr = MyModel()
ptr = ptr.to(device)

In [151]:
class BestModel:
    def __init__(self, path, initial_criterion):
        self.path = path
        self.criterion = initial_criterion
        
    def update(self, model, optimizer, criterion):
        self.criterion = criterion
        torch.save({'model_state': model.state_dict(), 'optimizer_state': optimizer.state_dict(), 'criterion': criterion}, self.path)
        
    def load_model_data(self):
        return torch.load(self.path)
    
    def restore(self, model, optimizer):
        model_data = self.load_model_data()
        model.load_state_dict(model_data['model_state'])
        optimizer.load_state_dict(model_data['optimizer_state'])

In [152]:
def train_model(epochs, model, optimizer, criterion, loaders, device, best_model, n_prints=1):
    print_every = len(loaders['train']) // n_prints
    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0
        
        for iteration, (xx, yy) in enumerate(loaders['train']):
            optimizer.zero_grad()
            xx, yy = xx.to(device), yy.to(device)
            out = model(xx)
            loss = criterion(out, yy)
            running_train_loss += loss.item()
            loss.backward()
            optimizer.step()
            
            if(iteration % print_every == print_every - 1):
                running_train_loss /= print_every
                print(f"Epoch {epoch}, iteration {iteration} training_loss {running_train_loss}")
                running_train_loss = 0.0
            
        with torch.no_grad():
            model.eval()
            running_corrects = 0
            running_total = 0
            running_loss = 0.0
            for xx, yy in loaders['validation']:
                batch_size = xx.size(0)
                xx, yy = xx.to(device), yy.to(device)

                out = model(xx)
                
                loss = criterion(out, yy)
                running_loss += loss.item()
                
                predictions = out.argmax(1)
                running_corrects += (predictions == yy).sum().item()
                running_total += batch_size
            
            mean_val_loss = running_loss / len(loaders['validation'])
            accuracy = running_corrects / running_total
            
            if accuracy > best_model.criterion:
                best_model.update(model, optimizer, accuracy)
            
            print(f"Epoch {epoch}, val_loss {mean_val_loss}, accuracy = {accuracy}")
    best_model.restore(model, optimizer)
                
                

In [153]:
import time
start = time.time()

In [154]:
optimizer = torch.optim.Adam(ptr.parameters(), lr=0.0003)
criterion = nn.CrossEntropyLoss()
best_model = BestModel('resnet_data',0.0)
train_model(5, ptr, optimizer, criterion,image_dataloaders, device, best_model=best_model, n_prints=5)

In [155]:
end = time.time()
print(end - start)

In [156]:
from torchvision.utils import make_grid

In [157]:
xx, yy = next(iter(image_dataloaders['validation']))

In [158]:
ptr.eval()
out = ptr(xx.cuda())
preds = out.argmax(1)
predicted_classes = [image_datasets['dev'].classes[c] for c in preds]
actual_classes = [image_datasets['dev'].classes[c] for c in yy]
imshow(make_grid(xx, nrow=4), fig_size=(8,8))
print(predicted_classes)
print(actual_classes)

In [159]:
all_preds = []
correct_preds = []
with torch.no_grad():
    ptr.eval()
    for xx, yy in image_dataloaders['test']:
        xx = xx.to(device)
        output = ptr(xx)
        all_preds.extend(output.argmax(1).tolist())
        correct_preds.extend(yy.tolist())

In [160]:
from sklearn import metrics

In [161]:
all_preds = np.asarray(all_preds)
correct_preds = np.asarray(correct_preds)

In [162]:
target_names = image_datasets['test'].classes
print(metrics.classification_report(correct_preds, all_preds,target_names=target_names))

In [163]:
confusion_matrix = metrics.confusion_matrix(correct_preds, all_preds)
pd.DataFrame(confusion_matrix, index=target_names, columns=target_names)

# 1.3

In [166]:
def grad(x,y):
    x = x.clone()
    y = y.clone()
    x.requires_grad_(True)
    f = 1 / (1 + torch.exp(torch.dot(x,y)))
    f.backward()
    return x.grad
x = torch.tensor([1.,5.,-3.,4.])
y = torch.tensor([-2.,-6.,4.,-5.])
grad(x,y)
